# Predicting Thoracic Pathologies 

## EDA + Preprocessing

In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
import tensorflow as tf
sns.set()

## Dataset

Our dataset is located at [ChestX-ray8 dataset](https://arxiv.org/abs/1705.02315). The dataset was originally created based on [this publication](https://nihcc.app.box.com/v/ChestXray-NIHCC/file/256057377774). This chest X-ray database (CXR8) was constructed due to addressing a need for hospital-scale radiological imaging presenting realistic clinical and methodological challenges, and present a public database for uses in disease analyses.

First, we can take a look at details of our patient data by checking the Data Entry csv file showing general characteristics of each subject's image. Most importantly we will see the finding labels, which are generated via NLP from radiologist reports. We can also see more details including patient age/sex, the view position of the image (i.e. Posterior-Anterior), and the original image's dimensions. 

In [ ]:
patient_data = pd.read_csv("../Data_Entry_2017_v2020.csv")
patient_data.head(5)

In [ ]:
patient_data['Finding Labels'].head(5)

In [ ]:
patient_data['Patient ID'].tail(1)

Our patient data totals to 112,120 total medical images for 30,805 unique patients. These images have been collected from patients spanning 1992 - 2015 from the clinical PACS database at NIHCC (National Institutes of Health Clinical Center) in which image labels are mined via NLP from radiology reports. The text-mined disease labels are expectedly >90% accuracy. Images are labeled as XXX_YYY.png, where XXX represents patient ID and YYY represents the CXR ID (or followup) of that patient.

Our finding labels include a total of 14 of the most common thoracic pathologies:

Atelectasis, Cardiomegaly, Effusion, Infiltration, Mass, Nodule, Penumonia, Penumothorax, Consolidation, Edema, Emphysema, Fibrosis, Pleural Thickening, and Hernia.

Using our first patient (57M) as an example, we can see their first CXR (00000001_000.png) which is labeled with cardiomegaly. 

<img src="../images/00000001_000.png" alt="U-net Image" width="300" align="middle"/>


## Considerations

We want to make sure that our machine learning model(s) recognize specific features and pathologies, rather than simply memorize patient-specific attributes. To ensure this, while splitting our dataset into training, testing, and validation sets we will make sure our all of the images of each patient belong to a single set (i.e. no patient should have one image in a training set and another image in a validation or testing set). 

This data is already divided accordingly to text files test_list.txt and train_val_list.txt associated with the CXR8 dataset as recommended by the authors. Using a structured split dataset as referenced by [a publication analyzing pruning impact on medical image classifiers](https://arxiv.org/pdf/2308.09180), we have three labeled datasets respecting the patient division structured with one-hot-encoded columns representing pathologies.

Each set also contains a column for Subject ID, Image ID, and for No Findings (i.e. none of the 14 pathologies show up). Before we explore the training set in particular to assess data distribution, summaries, and structure we make sure to remove the disease features outside the 14 pathologies being focused on. 


In [ ]:
# Patient training data
# Removing unneeded pathologies from training, validation, and test sets

pt_train = pd.read_csv("miccai2023_nih-cxr-lt_labels_train.csv")
pt_train = pt_train.drop(columns=['Pneumoperitoneum', 
                                  'Pneumoperitoneum',
                                  'Pneumomediastinum', 
                                  'Subcutaneous Emphysema',
                                  'Tortuous Aorta', 
                                  'Calcification of the Aorta'],
                        axis = 0)
pt_valid = pd.read_csv("miccai2023_nih-cxr-lt_labels_val.csv").drop(columns=['Pneumoperitoneum', 
                                  'Pneumoperitoneum',
                                  'Pneumomediastinum', 
                                  'Subcutaneous Emphysema',
                                  'Tortuous Aorta', 
                                  'Calcification of the Aorta'],
                        axis = 0)
pt_test = pd.read_csv("miccai2023_nih-cxr-lt_labels_test.csv").drop(columns=['Pneumoperitoneum', 
                                  'Pneumoperitoneum',
                                  'Pneumomediastinum', 
                                  'Subcutaneous Emphysema',
                                  'Tortuous Aorta', 
                                  'Calcification of the Aorta'],
                        axis = 0)

pt_train.head()
# pt_valid.head()
# pt_test.head()

In [ ]:
pt_train.info()

In [ ]:
pathology_cols = list(pt_train.drop(['id','subj_id'], axis = 1).keys())
pathology_cols

We can see that there are no null values in our dataset, and that our data types are standardized (one-hot-encoded as binary int64 values).

Now we'll focus on the class distributions for each type of pathology.

In [ ]:
# Count up the number of instances of each class (drop non-class columns from the counts)
class_counts = pt_train.sum().drop(['id','subj_id'])

for column in class_counts.keys():
    print(f"The class {column} has {pt_train[column].sum()} samples")

# Plotting pathology count distributions
sns.barplot(x = class_counts.values, y = class_counts.index, color='r')
plt.title('Distribution of Classes for Training Dataset', fontsize=15)
plt.xlabel('Number of Patients', fontsize=15)
plt.ylabel('Diseases', fontsize=15)
plt.show()

Based on these distributions, it is clear that we have a significantly imbalanced dataset showing a majority of images presenting with no findings. We can also see that hernias are the least represented pathology with only 130 total samples. Turning to literature, we can see that the [following publication](https://arxiv.org/pdf/2208.13365) addresses clinical data imbalances. In particular, it discusses possible avenues to address the distribution imbalance with majority of patients present without any of the 14 indicated pathologies.

### Ensuring no patient overlap

As mentioned previously, we do not want patient overlap--this way, our model is trained on specific pathological features rather than specific patient features. We can check for overlap between our training, validation and test sets:

In [ ]:
ids_train_set = set(pt_train.subj_id.values)
ids_valid_set = set(pt_valid.subj_id.values)
ids_test_set = set(pt_test.subj_id.values)
patient_overlap = list(ids_train_set.intersection(ids_valid_set))
patient_overlap_test = list(ids_test_set.intersection(ids_valid_set))
n_overlap = len(patient_overlap)
n_overlap_test = len(patient_overlap_test)
print(f'There are {n_overlap} Patient IDs in both the training and validation sets')
print(f'There are {n_overlap_test} Patient IDs in both the validation and test sets')

## Checking a single CXR


In [ ]:
# Looking at first image in depth

img_dir = '../images'

sample_img = pt_train.id[0]
raw_image = plt.imread(os.path.join(img_dir, sample_img))
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('Raw Chest X Ray Image')
print(f"The dimensions of the image are {raw_image.shape[0]} pixels width and {raw_image.shape[1]} pixels height, one single color channel")
print(f"The maximum pixel value is {raw_image.max():.4f} and the minimum is {raw_image.min():.4f}")
print(f"The mean value of the pixels is {raw_image.mean():.4f} and the standard deviation is {raw_image.std():.4f}")

### Image Preprocessing

We need to process images for our convolutional neural networks to train more effectively. The images should be normalized and scaled for the CNN to converge faster. Moreover, our limited dataset size (especially for representation on less common pathologies like hernias) can be effectively increased by data augmentation to reduce chances of overfitting. However, we do need to keep in mind that not all augmentation is appropriate--for example, flipping an image (across the Y axis) would place the heart on the wrong side of the patient, which would represent incorrect anatomy for our network to be trained on. 

Generally speaking, properly preprocessed images should help highlight our essential features while minimizing less relevant ones (ie related to noise, extreme brightness, etc). We start by looking at our first image in greater depth:

In [ ]:
# Looking at first image in depth

img_dir = '../images'

sample_img = pt_train.id[0]
raw_image = plt.imread(os.path.join(img_dir, sample_img))
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('Raw Chest X Ray Image')
print(f"The dimensions of the image are {raw_image.shape[0]} pixels width and {raw_image.shape[1]} pixels height, one single color channel")
print(f"The maximum pixel value is {raw_image.max():.4f} and the minimum is {raw_image.min():.4f}")
print(f"The mean value of the pixels is {raw_image.mean():.4f} and the standard deviation is {raw_image.std():.4f}")

Our images are largely 1024 x 1024 in size, but we don't have standardizes mean/StD values for pixel intensity. We can create a list of file paths and multi-hot label vectors, incorporate a tf.data.Dataset using `from_tensor_slices` and map out each image to perform per-image normalization. We can batch and prefetch the dataset for performance.

In [ ]:
image_files = pt_train['id'].values
labels = pt_train[pathology_cols].values
dataset = tf.data.Dataset.from_tensor_slices((image_files, labels))
# This dataset gives us .png + label pairs

In [ ]:
IMG_SIZE = (224, 224)
# DenseNet121 designed for ImageNet images of sisze 224x224, so we reduce size of our original 1024x1024

def load_and_preprocess_image(filename, label):
    # 1) Construct full path
    full_path = tf.strings.join([img_dir, filename], separator=os.path.sep)
    
    # 2) Load and decode the image
    image_bytes = tf.io.read_file(full_path)
    image = tf.io.decode_image(image_bytes, channels=3, expand_animations=False)
    
    # 3) Resize
    image = tf.image.resize(image, IMG_SIZE)
    
    # 4) Convert to float
    image = tf.cast(image, tf.float32)
    
    # 5) Samplewise normalization => mean=0, std=1
    mean, var = tf.nn.moments(image, axes=[0, 1, 2])  # entire image
    std = tf.sqrt(var)
    image = (image - mean) / tf.maximum(std, 1e-7) # avoid division by 0
    
    return image, label # returns normalized image + label

### Data Augmentation

We'll introduce very small random rotations, zooms, and brightness adjustments to give our smaller dataset a better chance in terms of generalization with our model.

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomContrast(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomRotation(0.05), 
    # No flipping to be safe
])

In [ ]:
BATCH_SIZE = 32
# Smaller input images means bigger batches in GPU memory? 
# if run out of GPU memory, drop to 16

train_dataset = (
    dataset
    .shuffle(buffer_size=len(pt_train), reshuffle_each_iteration=True) # randomize order of pairs 
    .map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE) # normalizes each image
    .map(lambda x, y: (data_augmentation(x, training=True), y),
       num_parallel_calls=tf.data.AUTOTUNE) # apply augments
    .cache() # if enough RAM, allows decoded image to not be repeatedly read from disk each epoch
    .batch(BATCH_SIZE) # grouping to 8 images/labels each
    .prefetch(tf.data.AUTOTUNE) # more efficient training
)

# we can iterate through dataset: for batch_images, batch_labels in train_dataset: ...
# or with a keras model, train on this dataset directly
# or manually fetch batches (below)

Now we can fetch batches and take a look at different images--we can see that our mean and standard deviations for pixels are 0 and 1 respectively. 

In [ ]:
# Fetch one batch
images_batch, labels_batch = next(iter(train_dataset))
print("Batch of images shape:", images_batch.shape)  # (8, 224, 224, 1)
print("Batch of labels shape:", labels_batch.shape)  # (8, 15)

# First image in this batch
sample_image = images_batch[0].numpy()  # shape: (320, 320, 1)
sample_label = labels_batch[0].numpy()  # shape: (14,)

print(f"Sample image stats: mean={sample_image.mean():.4f}, "
      f"std={sample_image.std():.4f}")

sns.set_style("white")
plt.imshow(sample_image.squeeze(), cmap="gray")
plt.colorbar()
plt.title("Sample Standardized Chest X-ray")
plt.show()

print("Corresponding label vector:", sample_label)

### Data Imbalance Considerations

As previously discussed, we need to account for data imbalances--particularly due to the fact that a significant majority of CXRs present with no findings. In order to create an effective model or CNN, we need to consider weighted loss functions to address this imbalance. We want to train our model with an evenly balanced dataset so positive and negative training cases are contributing equally to the loss. 

Using a normal cross-entropy loss function with an unbalanced dataset (like the one we have) results in more prioritization in the majority class (ie negative here) as it contributes more to the loss. 

One way to change this contribution inequality is multiplying each example from each class by class-specific weight factors so overall contribution of each class is the same.

After applying these weightings, we implement a loss function:
$$\mathcal{L}_{cross-entropy}^{w}(x) = - (w_{p} y \log(f(x)) + w_{n}(1-y) \log( 1 - f(x) ) ).$$


In [ ]:
def compute_class_freqs(labels):
    # pos/neg freq for each class given labels
    N = labels.shape[0]
    positive_frequencies = np.sum(labels, axis = 0) / N
    negative_frequencies = 1.0 - positive_frequencies
    return positive_frequencies, negative_frequencies

In [ ]:
freq_pos, freq_neg = compute_class_freqs(labels)
print("Positive frequencies:", freq_pos)
print("Negative frequencies:", freq_neg)

We can look at our positive vs negative contribution ratios per each pathology:

In [ ]:
x = np.arange(len(pathology_cols))
bar_width = 0.4

# Plot pos frequencies
plt.bar(
    x - bar_width/2,        # Shift the bars to the left by half the bar width
    freq_pos,
    width=bar_width,
    label='Positive',
)

# Plot neg frequencies
plt.bar(
    x + bar_width/2,        # Shift the bars to the right by half the bar width
    freq_neg,
    width=bar_width,
    label='Negative',
)
plt.xticks(x, pathology_cols, rotation=45, ha='right')
plt.ylabel('Frequency')
plt.title('Positive vs. Negative Frequencies for Each Pathology')
plt.legend()

It's clear here that there's a significantly higher contribution by negative cases than positive cases, with the exception of No Finding (representing normal).

In [ ]:
pos_weights = freq_neg
neg_weights = freq_pos
pos_contribution = freq_pos * pos_weights 
neg_contribution = freq_neg * neg_weights

In [ ]:
# Plot pos frequencies
plt.bar(
    x - bar_width/2,        # Shift the bars to the left by half the bar width
    pos_contribution,
    width=bar_width,
    label='Positive',
)

# Plot neg frequencies
plt.bar(
    x + bar_width/2,        # Shift the bars to the right by half the bar width
    neg_contribution,
    width=bar_width,
    label='Negative',
)
plt.xticks(x, pathology_cols, rotation=45, ha='right')
plt.ylabel('Frequency')
plt.title('Positive vs. Negative Frequencies for Each Pathology')
plt.legend()

We create our `get_weighted_loss` function that calculates weighted loss per batch--for multi-class loss, we add up average loss for each individual class. To aavoid possible numerical errors (ie if predicted value is 0), we add a small value $\epsilon$ to predicted values prior to taking their log values.

In [ ]:
def get_weighted_loss(pos_weights, neg_weights, episilon = 1e-7):
    # return weighted loss function given pos/neg weights (arr for each class, size)
    def weighted_loss(y_true, y_pred):
        # return weighted loss value given tensors of true labels/predicted labels
        loss = 0.0
        for i in range(len(pos_weights)):
            pos_loss = -pos_weights[i] * y_true[:, i] * tf.math.log(y_pred[:, i] + episilon)
            neg_loss = -neg_weights[i] * (1.0 - y_true[:, i]) * tf.math.log(1.0 - y_pred[:, i] + episilon)
            loss += tf.math.reduce_mean(pos_loss + neg_loss)
        return loss
    return weighted_loss

Now we prepare our validation and test datasets with a `tf.data` pipeline, as we did with our training set. 

In [ ]:
val_image_files = pt_valid['id'].values
val_labels      = pt_valid[pathology_cols].values
val_dataset     = tf.data.Dataset.from_tensor_slices((val_image_files, val_labels))

val_dataset = (
    val_dataset
    .map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)


In [ ]:

test_image_files = pt_test['id'].values
test_labels      = pt_test[pathology_cols].values
test_dataset     = tf.data.Dataset.from_tensor_slices((test_image_files, test_labels))

test_dataset = (
    test_dataset
    .map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)



We can start using a pre-trained model loaded directly from Keras:

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K


base_model = DenseNet121(weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False
    
    #freezing layers: fine-tune CXRs and only train last few layers + new head

x = base_model.output

# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)

# and a logistic layer
predictions = Dense(len(pathology_cols), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss=get_weighted_loss(pos_weights, neg_weights))


In [ ]:
# LR Scheduler so LR adapts when validation loss plateaus
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,          # reduce LR by a factor of 10
    patience=2,          # wait 2 epochs of no improvement
    verbose=1
)

# Early Stopping when validation doesn't improve
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # or a relevant metric
    patience=3,          # how many epochs to wait before stopping
    restore_best_weights=True
)

In [ ]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5
    callbacks = [lr_schedule, early_stop]

)


In [ ]:
model.fit(
    train_dataset,
    test_data = test_dataset,
    epochs = 5,
    callbacks = [lr_schedule, early_stop]
)